In [12]:
import json
with open('ebird_config.json') as f:
    data = json.load(f)
    recent_species_observations = data['recent_species_observations']
    ebird_token = data['ebird_token']
    gmap_api_key = data['gmap_api_key']
# recent_species_observations

'https://api.ebird.org/v2/data/nearest/geo/recent/evegro?lat=38.90&lng=-77.07&back=2&includeProvisional=true&maxResults=50'

In [13]:
import requests
headers = {}
headers['X-eBirdApiToken'] = ebird_token
response = requests.get(recent_species_observations, headers=headers)
res_json = response.json()

In [17]:
# res_json

In [18]:
sightings = list()
for sighting in res_json:
    sightings.append((sighting['lat'], sighting['lng']))

In [16]:
import gmaps
gmaps.configure(api_key=gmap_api_key)
fig = gmaps.figure()
sightings_layer = gmaps.symbol_layer(sightings)
fig.add_layer(sightings_layer)
fig

Figure(layout=FigureLayout(height='420px'))